# Simularium Conversion Tutorial : MEDYAN Data

In [1]:
from IPython.display import Image
import numpy as np
from simulariumio.medyan import MedyanConverter, MedyanData, MedyanAgentInfo
from simulariumio import MetaData
from simulariumio.filters import TranslateFilter

This notebook provides example python code for converting your own simulation trajectories into the format consumed by the Simularium Viewer. It creates a .simularium JSON file which you can drag and drop onto the viewer like this:

![title](img/drag_drop.gif)

***
## Prepare your spatial data

The Simularium `MedyanConverter` consumes discrete cell data from MEDYAN. 

The converter requires a `MedyanData` object as a parameter.

`MedyanData` contains the following:
* **meta_data** : `MetaData`
    * An object containing metadata for the trajectory including box size, scale factor, and camera defaults        
* **path_to_snapshot** : *string*
    * A string path to the MEDYAN snapshot.traj output file          
* **filament_agent_info** : *Dict\[int, MedyanAgentInfo\] (optional)*
    * A dict mapping MEDYAN type ID for filaments to info (name, radius) for filament agents          
* **linker_agent_info** : *Dict\[int, MedyanAgentInfo\] (optional)*
    * A dict mapping MEDYAN type ID for linkers to info (name, radius) for linker agents          
* **motor_agent_info** : *Dict\[int, MedyanAgentInfo\] (optional)*
    * A dict mapping MEDYAN type ID for motors to info (name, radius) for motor agents          
* **draw_fiber_points** : *bool (optional)*
    * in addition to drawing a line for each fiber, also draw spheres at every other point along it?
    * Default: False         
* **plots** : *List\[Dict\[str, Any\]\] (optional)*
    
`MetaData` contains the following:
* **box_size** : *np.ndarray (shape = \[3\])*
    * A numpy ndarray containing the XYZ dimensions of the simulation bounding volume
        * *for now we only support rectangular bounds, but will be adding support for spheres soon*
* **camera_defaults**: `CameraData` (optional)
    * camera's initial settings which it also returns to when reset
* **scale_factor** : float (optional)
    * A multiplier for the scene, use if visualization is too large or small
    * Default: 1.0
    
`CameraData` contains the following:
* **position** : *np.ndarray (shape = \[3\]) (optional)*
    * 3D position of the camera itself
    * Default: np.array(\[0.0, 0.0, 120.0\])
* **look_at_position**: *np.ndarray (shape = \[3\]) (optional)*
    * position at which the camera looks
    * Default: np.zeros(3)
* **up_vector**: *np.ndarray (shape = \[3\]) (optional)*
    * the vector that defines which direction is "up" in the camera's view
    * Default: np.array(\[0.0, 1.0, 0.0\])
* **fov_degrees**: *float (optional)*
    * the angle defining the extent of the 3D world that is seen from bottom to top of the camera view
    * Default: 75.0

In [2]:
box_size = 1000.0
scale = 0.1

example_data = MedyanData(
    meta_data=MetaData(
        box_size=np.array([box_size, box_size, box_size]),
        scale_factor=scale,
    ),
    path_to_snapshot="../simulariumio/tests/data/medyan/snapshot.traj",
    filament_agent_info={
        0: MedyanAgentInfo(
            name="Filament",
            radius=5.0,
        ),
    },
    linker_agent_info={
        0: MedyanAgentInfo(
            name="LinkerA",
            radius=8.0,
        ),
        1: MedyanAgentInfo(
            name="LinkerB",
            radius=8.0,
        ),
        2: MedyanAgentInfo(
            name="LinkerC",
            radius=8.0,
        ),
    },
    motor_agent_info={
        1: MedyanAgentInfo(
            name="Motor",
            radius=2.0,
            draw_endpoints=True,
            endpoint_radius=10.0,
        ),
    },
)

## Convert and save as .simularium JSON file

Once your data is shaped like in the `example_data` object, you can use the converter to generate the file at the given path:

(since MEDYAN coordinates are all positive, use a `TranslateFilter` to center the data in the viewer.)

In [3]:
c = MedyanConverter(example_data)
translation_magnitude = -(box_size * scale) / 2
filtered_data = c.filter_data([
    TranslateFilter(
        translation_per_type_id={},
        default_translation=translation_magnitude * np.ones(3)
    ),
])
c.write_external_JSON(filtered_data, "example_medyan")

Reading MEDYAN Data -------------
Filtering: translation -------------
Writing JSON (external)-------------
Converting Trajectory Data -------------
saved to example_medyan.simularium


## Visualize in the Simularium viewer

In a supported web-browser (Firefox or Chrome), navigate to https://simularium.allencell.org/ and import your file into the view.